In [3]:
from collections import OrderedDict
from bokeh.models import ColumnDataSource, HoverTool
import bokeh.plotting as bk
import numpy as np
from math import pi
import pandas as pd

data = pd.read_csv("data/seattle_incidents_summer_2014.csv")
grouped = data.groupby(data["Offense Code"])

In [4]:
bk.output_notebook()



Loading BokehJS ...

In [61]:
# data.head()

In [5]:
counts = data["Offense Code"].value_counts()

mean_counts = counts / 3
counts_jun = data["Offense Code"][data["Month"] == 6].value_counts()
counts_jul = data["Offense Code"][data["Month"] == 7].value_counts()
counts_aug = data["Offense Code"][data["Month"] == 8].value_counts()

joint = pd.concat([mean_counts, counts_jun, counts_jul, counts_aug], 
                  axis=1, join_axes=[mean_counts.index])

joint.columns = ["c_mean", "c_jun", "c_jul", "c_aug"]

In [22]:
# TOOLS = "box_zoom,box_select,crosshair,resize,reset,hover"


x = range(1, len(joint) + 1)
codes = joint.index

TOOLS = "pan,box_zoom,resize,wheel_zoom,reset,hover,save"
p=bk.figure(title="Summer 2014 crime breakdown in Seattle", tools=TOOLS,
         plot_height=300, plot_width=900, )

source_mean = ColumnDataSource(data=dict(rank=x,
                                         count=joint.c_mean,
                                         code=codes))
source_jun = ColumnDataSource(data=dict(rank=x,
                                        count=joint.c_jun,
                                        code=codes))
source_jul = ColumnDataSource(data=dict(rank=x,
                                        count=joint.c_jul,
                                        code=codes))
source_aug = ColumnDataSource(data=dict(rank=x,
                                        count=joint.c_aug,
                                        code=codes))

p.line(x, joint.c_mean, color="#222222", line_width=3, name="foo", 
       source=source_mean, legend="Monthly mean")
p.line(x, joint.c_jun, color="#aa2222", line_width=3, name="foo2", 
       source=source_jun, legend="June incidents")
p.line(x, joint.c_jul, color="#22aa22", line_width=3, name="foo3", 
       source=source_jul, legend="August incidents")
p.line(x, joint.c_aug, color="#2222aa", line_width=3, name="foo4", 
       source=source_aug, legend="July incidents")

hover = p.select(dict(type=HoverTool))

hover.tooltips = OrderedDict([
        ("Rank", "@rank"),
        ("Incidents", "@count"),
        ("Offense Code", "@code")
    ])

bk.show(p)